# Aula 7 - Exercícios - Pandas Parte 3 (Merge e Join)

In [1]:
# Importe a biblioteca pandas 
import pandas as pd

### 7.1 Escreva uma função para fazer o merge dos dataframes de população e de PIB de 2017. A função deve receber como parâmetro o caminho para cada um dos dataframes. Na operação de merge, o pandas renomeará as colunas municipio para municipio_x e municipio_y. Mantenha esses dois nomes de coluna, pois eles serão usados em exercícios posteriores.

#### Dica 1: Você já resolveu uma parte deste problema nos exercícios 6.2 e 6.3. Reaproveite o código copiando e colando aqui, ou refaça-o.
#### Dica 2: A coluna que contem o código IBGE do município deve ter o mesmo formato (string) e a mesma quantidade de caracteres nas duas tabelas.
#### Dica 3: Converta o codigo do município para string ao carregar o dataframe de população utilizando o mapeamento de tipos dtype={'cod_munic': str, 'cod_uf': str}. Assim o codigo do municipio não perde os zeros a esquerda.
#### Dica 4: Concatene o codigo da UF com o código do município usando o operador + e salve em outra coluna.
#### Dica 5: Use o código apresentado na aula para carregar o dataframe do PIB de 2017, especifique o dtype da coluna que contém o codigo do municipio.
#### Dica 6: Renomeie as colunas apropriadamente antes de fazer o merge. A coluna com o nome do municipio deve se chamar municipio (sem acento) e a coluna com o codigo ibge de 7 caracteres cod_ibge7. Veja o exemplo: {'Unnamed: 0': 'nivel', 'Unnamed: 1': 'cod_ibge7', 'Unnamed: 2': 'municipio'}. 
#### Dica 7: Depois que os dois dataframes tiverem o codigo ibge (com valores equivalentes  e mesmo tipo de dados) você pode fazer o merge.


In [ ]:
def merge_pop_pib(path_pop, path_pib):
    """retorne um dataframe resultante do merge dos dados do pib e da população
    """
    ### BEGIN SOLUTION
    df_pop = pd.read_excel(path_pop, sheet_name="Municipios", dtype={'cod_munic': str, 'cod_uf': str})
    # Concatena o codigo da UF com o codigo do municipio
    df_pop['cod_ibge7'] = df_pop['cod_uf'] + df_pop['cod_munic']
    df_pib = pd.read_excel(path_pib, skiprows=3, skipfooter=1, dtype={'Unnamed: 1': str})
    df_pib.rename(columns={'Unnamed: 0': 'nivel', 'Unnamed: 1': 'cod_ibge7', 
                           'Unnamed: 2': 'municipio'}, inplace=True)

    return pd.merge(df_pop, df_pib, on='cod_ibge7')
    ### END SOLUTION


In [ ]:
## Faça seus testes aqui
path_pib = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/pib/pib_municipios.xlsx'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'


In [ ]:
# Validação
path_pib = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/pib/pib_municipios.xlsx'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'

entradas = [[path_pop, path_pib]]
saidas = [{'pop2017': 25437, '2017': 498864}]
validate(merge_pop_pib, entradas, lambda x: x.loc[0,['pop2017', '2017']].to_dict(), saidas, "7.1", False)

### 7.2 Escreva uma função para calcular o PIB percapita  de 2017 de cada município do Brasil, considerando o dataframe resultante do exercício anterior.
#### Nomeie a coluna do PIB Percapita com 'pib_percapita' e arredonde o resultado para uma casa decimal usando a função ```round```.
#### Dica: Use a função ```round``` dentro de uma função ```apply```.

In [ ]:
def pib_percapita(pib_pop):
    """retorne um dataframe contendo o PIB percapita dos municípios.
    """
    ### BEGIN SOLUTION
    pib_pop['pib_percapita'] = pib_pop['2017']/pib_pop['pop2017']
    pib_pop['pib_percapita'] = pib_pop['pib_percapita'].apply(lambda x: round(x, 1))
    return pib_pop
    ### END SOLUTION


In [ ]:
## Faça seus testes aqui



In [ ]:
# Validação
# Considerando a função merge_pop_pib do exercicio 7.1
df_pib_pop = merge_pop_pib(path_pop, path_pib)

entradas = [[df_pib_pop]]
saidas = [19.6]
validate(pib_percapita, entradas, lambda x: x.loc[0,'pib_percapita'], saidas, "7.2", False)

### 7.3 Escreva uma função para retornar os registros dos 10 maiores PIBs percapita do brasil (de 2017) a partir do dataframe do exercício anterior. 
#### Escreva uma função genérica que receba como parâmetro um dataframe e o nome da coluna de ordenação.
#### Dica 1: Use o método sort_values para ordenar o dataframe. Para uma ordenação descendente use ascending=False.
#### Dica 2: Use o atributo iloc para retornar todas as colunas dos 10 primeiros registros.

In [ ]:
def top10(df, col):
    """retorne os top 10 registros do dataframe df ordenados por col
    """
    ### BEGIN SOLUTION
    return df.sort_values(by=col, ascending=False).iloc[:10,:]
    ### END SOLUTION


In [ ]:
## Faça seus testes aqui
df_pib_pop = merge_pop_pib(path_pop, path_pib)
pib_percapita_2017 = pib_percapita(df_pib_pop)


In [ ]:
# Validação
df_pib_pop = merge_pop_pib(path_pop, path_pib)
pib_percapita_2017 = pib_percapita(df_pib_pop)

test_data = {'cod_ibge7': {3677: '3536505', 3547: '3524709'},
             'municipio_x': {3677: 'Paulínia', 3547: 'Jaguariúna'},
             'uf': {3677: 'SP', 3547: 'SP'},
             'pib_percapita': {3677: 344.8, 3547: 209.3}}
test_data_y = {'cod_ibge7': {3677: '3536505', 3547: '3524709'},
             'municipio_y': {3677: 'Paulínia', 3547: 'Jaguariúna'},
             'uf': {3677: 'SP', 3547: 'SP'},
             'pib_percapita': {3677: 344.8, 3547: 209.3}}

entradas = [[pib_percapita_2017, 'pib_percapita']]
saidas = [test_data]
val1 = validate(top10, entradas, lambda x: x[['cod_ibge7', 'municipio_x', 'uf', 'pib_percapita']].iloc[[0,9]].to_dict(), saidas, "7.3", False)
if not val1:
    saidas = [test_data_y]
    val2 = validate(top10, entradas, lambda x: x[['cod_ibge7', 'municipio_y', 'uf', 'pib_percapita']].iloc[[0,9]].to_dict(), saidas, "7.3", False)
    print(val2)
else:
    print(val1)

### 7.4 Escreva uma função para fazer o merge do dataset de morbidade consolidado com o dataset de população
#### O dataset da população deve ficar a esquerda e o dataset da morbidade a direita no resultado.
#### Dica 1: No dataset de morbidade existem valores NA representados por caracteres específicos em várias colunas do dataframe. Descubra quais são esses caracteres e especifiquei no argumento na_values. Passe nesse argumento uma lista como os caracteres que devem ser interpretados como NA.
#### Dica 2: No dataset de morbidade, especifique o separador, o caracter de números decimais e o tipo do dado do cod_ibge6 como str. Ou seja, dtype={'cod_ibge6': str}.
#### Dica 3: No dataset de população, converta o cod_munic e cod_uf para str no momento de carregar o dataset de população, usando novamente o argumento dtype. 
#### Dica 4: No dataset de população, construa o código ibge de 6 dígitos do município a partir das colunas ```cod_uf``` e ```cod_munic```.


In [ ]:
def merge_morbidade_pop(path_pop, path_morbidade):
    """retorne o merge dos datasets de população e morbidade.
    """
    ### BEGIN SOLUTION
    df_pop = pd.read_excel(path_pop, sheet_name="Municipios", dtype={'cod_munic': str, 'cod_uf': str})
    # Concatena o codigo da UF com o codigo do municipio
    df_pop['cod_ibge6'] = df_pop['cod_uf'] + df_pop['cod_munic'].apply(lambda x: x[:-1])
    df_morbidade = pd.read_csv(path_morbidade, sep=';', decimal=',', dtype={'cod_ibge6': str}, 
                               na_values=['-', '...'])
    return pd.merge(df_pop, df_morbidade, on='cod_ibge6') 
    ### END SOLUTION


In [ ]:
## Faça seus testes aqui
path_morbidade = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/morbidade/consolidado/morbidade.csv'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'


In [ ]:
path_morbidade = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/morbidade/consolidado/morbidade.csv'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'
test_data = {'Internações': 149, 'AIH_aprovadas': 149, 'Valor_total': 101282.26}

entradas = [[path_pop, path_morbidade]]
saidas = [{'Internações': 149, 'AIH_aprovadas': 149, 'Valor_total': 101282.26}]
val1 = validate(merge_morbidade_pop, entradas, lambda x: x[(x['mes']=='Abr') & (x['cod_munic']=='00015')].loc[:,['Internações', 'AIH_aprovadas', 'Valor_total']].to_dict(orient='records')[0], saidas, "7.4", False)
if not val1:
    val2 = validate(top10, entradas, lambda x: x.iloc[0, 8:11].to_dict(), saidas, "7.4", False)
    print(val2)
else:
    print(val1)